In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import utilities # codeTimer context manager and saving/loading utilities.
import data_preparation # Load dataset and build required matrices.
import factorisation # WALS factorisation.
import recommender # Recommender system.

### Loading dataset and creating recommender system

In [3]:
np.random.seed(17)

mov, rat, rat_test = data_preparation.importDataset()
rec = recommender.recommenderSystem(mov, rat, rat_test)
print("Prediction MAE: {}".format(rec.predictionError()))

The dataframe contains 610 users and 9721 items.
Prediction MAE: 46.3893943883063


In [4]:
# The a pre-trained recommander system can be loaded with the following line.

#utilities.loadRecSys(rec, "rec.npz")

### Explorative analysis

In [ ]:
np.count_nonzero(rec.R)
sparsity = np.count_nonzero(rec.R)/np.prod(rec.R.shape)
print("Matrix sparsity: {}%".format(sparsity*100))

In [ ]:
# Distribution of number of ratings.
from matplotlib.ticker import MaxNLocator
plt.rcParams.update({'font.size': 13})

n_ratings = [np.count_nonzero(i) for i in rec.R]


fig, ax = plt.subplots()

ax.hist(n_ratings, bins = 150)

ax.set_title("Number of ratings histogram")
ax.set_ylabel("Users")
y_vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in y_vals/np.shape(rec.R)[0]])
ax.set_xlim(0, 600)
ax.set_xlabel("Number of ratings")
ax.xaxis.set_major_locator(MaxNLocator(integer = True))

plt.savefig("n_ratings.pdf", transparent = True)
plt.show()

In [ ]:
# Counting number ratings.
rates = rec.R.flatten()
rates = rates[rates != 0]
unique, counts = np.unique(rates, return_counts = True)

fig, ax = plt.subplots()
ax.bar(unique, counts / sum(counts), width = 0.4)

ax.set_title("Distribution of ratings")

y_vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in y_vals])

ax.set_xlabel("Rating")
ax.xaxis.set_major_locator(MaxNLocator(integer = False))
plt.rcParams['xtick.labelsize'] = 11
ax.set_xticks(list(unique))

plt.savefig("ratings_dist.pdf", transparent = True)
plt.show()

### Selecting best regression coefficient with CV

In [ ]:
np.random.seed(17)

reg_lambda = [0.05, 0.15, 0.30]

n_folds = 4
n_iter = 4

# reg_lambda is required to be a list.
with utilities.codeTimer("Best regression lambda CV"):
    best_lambda, errors = rec.bestLambdaCV(n_folds, n_iter, reg_lambda)

In [ ]:
# Plotting results.

fig, ax = plt.subplots()
x = [i + 1 for i in range(n_iter)]
for i in range(len(reg_lambda)):
    ax.plot(x, errors[i][1])

ax.set_title("kFoldCV error")
ax.set_ylabel("MAE")
ax.set_xlabel("Iteration")
ax.xaxis.set_major_locator(MaxNLocator(integer = True))
ax.legend(reg_lambda, title = r"$\lambda$ values")

plt.savefig("TestErrorCV.pdf", transparent = True)
plt.show()

### Factorisation

In [ ]:
# Expect ~ 52 min for the execution on the whole dataset with 10 iterations.
#reg_lambda = best_lambda
reg_lambda = 0.15
n_iter = 10

with utilities.codeTimer("WALS factorisation"):
    train, test = rec.performFactorisation(reg_lambda, n_iter)

In [ ]:
# Plotting results.
fig, ax = plt.subplots()
x = [i + 1 for i in range(len(test))]
ax.plot(x, test)

ax.set_title("WALS factorisation")
ax.set_ylabel("MAE")
#ax.set_ylim(0, 0.9)
ax.set_xlabel("Iteration")
ax.xaxis.set_major_locator(MaxNLocator(integer = True))

plt.savefig("WALS_train.pdf", transparent = True)
plt.show()

In [ ]:
# Once the recommender has been trained, it can be saved with the
# following line.

#utilities.saveRecSys(rec, "rec.npz")

### Recommendation

In [5]:
def recommend(rec_system, user_id):
    return rec_system.answerQuery(user_id)
        
def bestRated(rec_system, user_id):
    user_movies = rec_system.getUserMovies(user_id)
    return user_movies.sort_values(by = "Rating", ascending = False)

In [ ]:
user_id = 2
recommend(rec, user_id).head(10)

In [ ]:
bestRated(rec, user_id).head(10)

### Similar items
Some suggestions:
* 911: Star Wars Episode VI
* 3638: The Lord of the Rings: The Fellowship of the Ring
* 957: The Shining
* 474: Blade Runner

In [ ]:
rec.suggestSimilar(957)

### New user recommendation

In [21]:
np.random.seed(17)

new_user, new_user_id = rec.generateNewUser(50)
np.shape(rec.R)

(613, 9721)

In [22]:
new_user_id
reg_lambda = 0.15

In [23]:
with utilities.codeTimer("New user factorisation"):
    rec.addNewUser(new_user, reg_lambda)
np.shape(rec.R)

Executed 'New user factorisation'.  Elapsed time: 0.700787s


(614, 9721)

In [24]:
recommend(rec, new_user_id).head(10)

,MovieID,Prediction,Title,Genres,AVG_Rating
1050,1060,5.94,Batman Returns (1992),Action|Crime,3.025000
1995,2014,5.72,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy,3.183333
926,935,5.16,Harold and Maude (1971),Comedy|Drama|Romance,4.288462
3886,3956,4.97,"Transporter, The (2002)",Action|Crime,3.519231
4609,4705,4.93,Battle Royale (Batoru rowaiaru) (2000),Action|Drama|Horror|Thriller,3.948276
2198,2222,4.91,"Fistful of Dollars, A (Per un pugno di dollari...",Action|Western,3.986486
2241,2265,4.91,"Insider, The (1999)",Drama|Thriller,3.700000
2207,2231,4.90,"Straight Story, The (1999)",Adventure|Drama,3.928571
6076,6209,4.81,X-Men: The Last Stand (2006),Action|Sci-Fi|Thriller,3.281250
6484,6625,4.72,"Mist, The (2007)",Horror|Sci-Fi,3.653846


In [25]:
bestRated(rec, new_user_id).head(10)

,MovieID,UserID,Genres,Title,Rating
94890,804,613,Crime|Drama|Romance,Normal Life (1996),5.0
94891,973,613,Action|Adventure|Animation|Sci-Fi,Akira (1988),5.0
94902,2380,613,Drama,"Emperor and the Assassin, The (Jing ke ci qin ...",5.0
94929,7394,613,Action|Adventure|Thriller,"Expendables, The (2010)",5.0
94914,5076,613,Horror|Sci-Fi,From Beyond (1986),4.5
94923,6892,613,Drama|Fantasy,Krabat (2008),4.5
94920,6195,613,Fantasy|Horror|Thriller,Silent Hill (2006),4.5
94926,7319,613,Action|Adventure|Drama|Mystery|Thriller,"Losers, The (2010)",4.5
94925,7292,613,Action|Comedy|Romance,"Bounty Hunter, The (2010)",4.5
94917,5968,613,Action|Crime|Drama|Thriller,Pusher II: With Blood on My Hands (2004),4.0


### Cold start problem
If a new user has rated less than 10 movies, the most popular and unseen movies will be recommended.

In [ ]:
np.random.seed(17)

new_user, new_user_id = rec.generateNewUser(8)
np.shape(rec.R)

with utilities.codeTimer("New user factorisation"):
    rec.addNewUser(new_user, reg_lambda)

In [ ]:
recommend(rec, new_user_id).head(10)